# Delta Lake - Cenário e Operações

Este notebook descreve o uso do Delta Lake com Apache Spark.

## 🗂️ Modelo ER e Fonte de Dados

Usaremos uma tabela `usuarios` com o seguinte esquema:

- `id` (int)
- `nome` (string)
- `idade` (int)
- `status` (string)

A fonte de dados utilizada é um arquivo CSV público com dados de usuários fictícios.

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder     .appName("DeltaLakeExample")     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")     .getOrCreate()


In [ ]:

df = spark.read.csv("data/usuarios.csv", header=True, inferSchema=True)
df.show()


## 💾 Criando a tabela Delta

In [ ]:

df.write.format("delta").mode("overwrite").save("/tmp/delta/usuarios")


## ➕ Comando INSERT

In [ ]:

from pyspark.sql import Row

new_users = [Row(id=4, nome="Carlos", idade=22, status="ativo")]
spark.createDataFrame(new_users).write.format("delta").mode("append").save("/tmp/delta/usuarios")


## ✏️ Comando UPDATE

In [ ]:

from delta.tables import DeltaTable
delta_table = DeltaTable.forPath(spark, "/tmp/delta/usuarios")
delta_table.update("idade < 18", {"idade": "18"})


## ❌ Comando DELETE

In [ ]:

delta_table.delete("status = 'inativo'")
